In [5]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [6]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [7]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* name - The name of the country.
* area - The total land and sea area of the country.
* population - The country's population.
* population_growth- The country's population growth as a percentage.
* birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
* death_rate - The country's death rate, or the number of death a year per 1,000 people.
* area- The country's total area (both land and water).
* area_land - The country's land area in square kilometers.
* area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and look for any outlier countries.

In [13]:
#calculate some summary statistics and look for any outlier countries.

In [14]:
%%sql
SELECT MIN(population), 
        MAX(population), 
        MIN(population_growth),
        MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


It's easy to see from the last statement that there is some data that needs to be cleaned.

* there's a country with a population of 0
* there's a country with a population of 7256490011 (close to the entire world population!)

Lets us zoom in on just these countries.

In [15]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population)
                    FROM facts)
;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [16]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts)
;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Looks like the dataset contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

Let's calculate some averages:

In [17]:
%%sql
SELECT  CAST(AVG(population) AS INT) AS "Average Population",
        ROUND(AVG(area),2) AS "Average Area"
FROM    facts
WHERE   population < (SELECT MAX(population)
                    FROM facts)
        AND population > (SELECT MIN(population)
                    FROM facts)
;

 * sqlite:///factbook.db
Done.


Average Population,Average Area
32377011,582949.85


Now we can find countries that are densely populated. We'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [18]:
%%sql
SELECT *
FROM facts 
WHERE population > (SELECT  CAST(AVG(population) AS INT) AS "Average Population"
                    FROM    facts
                    WHERE   population < (SELECT MAX(population)
                                        FROM facts)
                            AND population > (SELECT MIN(population)
                                        FROM facts)
                   )
      AND
      area < (SELECT  ROUND(AVG(area),2) AS "Average Area"
              FROM    facts
              WHERE   population < (SELECT MAX(population)
                                    FROM facts)
                        AND population > (SELECT MIN(population)
                        FROM facts)
             )
;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
88,ke,Kenya,580367,569140,11227,45925301,1.93,26.4,6.89,0.22
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46


Here are some next steps for to explore:

- Which countries will add the most people to their population next year?
- Which countries have a higher death rate than birth rate?

Which countries have the highest ratios of water to land?

In [29]:
%%sql
SELECT *
FROM facts
WHERE (area_land > 0) AND (area_water > 0) AND (population > 0)
ORDER BY (CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT)) DESC

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46


Which countrie(s) have more water than land?

In [32]:
%%sql
SELECT name, area_water, area_land, population
FROM facts
WHERE (area_land > 0) AND (area_water > 0) AND (population > 0) AND (area_water > area_land)
;

 * sqlite:///factbook.db
Done.


name,area_water,area_land,population
Virgin Islands,1564,346,103574
